In [ ]:
!pip install auto-gptq torch

In [1]:
# Select model.
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

In [2]:
# Select dataset for calibration.
# Choosing something that is as close as possible to your target use case is best.
# Here, we selected a generic chat dataset.
DATASET_ID = "HuggingFaceH4/ultrachat_200k"

# Best practice is to use 4096, but 2048 can be good enough.
MAX_SEQ_LEN = 4096

# 1024 samples is usually good enough, moving to 2048 can help sometimes.
# Note: the bigger this is, the more memory and the longer the runtime.
NUM_SAMPLES = 2048

In [3]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Pre-process your dataset.
# Its a good idea to use the chat template.
def preprocess(example):
    return {"text": tokenizer.apply_chat_template(
        example["messages"], tokenize=False,
    )}

dataset = load_dataset(DATASET_ID, split="train_sft")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
ds = dataset.shuffle().select(range(NUM_SAMPLES))
ds = ds.map(preprocess)

# BE CAREFUL WITH THE TOKENIZER
#   apply_chat_template already adds the bos_token
#   so we set add_special_token to false
examples = [
    tokenizer(
        example["text"],
        padding=False, max_length=MAX_SEQ_LEN, truncation=True, add_special_tokens=False
    ) for example in ds
]

/home/rshaw/.pyenv/versions/3.10.14/envs/auto-gptq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 2048/2048 [00:00<00:00, 4677.82 examples/s]


In [4]:
# Confirm the input tokens look right.
# Note: its important that the chat template be applied.
# Note: careful that you don't end up with 2 <|begin_of_text|> tokens!
print(tokenizer.decode(examples[0]["input_ids"]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Can you describe the top open-world games and how they differ from linear games?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I can describe the top open-world games and how they differ from linear games. 

open-world games allow players to explore a vast, interconnected world, complete with various side quests, activities, and non-linear storylines. These games are designed to provide a sense of freedom and allow players to make meaningful choices that impact the gameplay experience. Some of the most popular open-world games include:

1. The witcher 3: wild hunt - a fantasy rpg where players control geralt of rivia, a monster hunter who travels a vast open world filled with diverse environments, characters, and side quests.

2. Grand theft auto v - an action-adventure game where players control multiple protagonists who explore a fictionalized version of los angeles, engaging in various criminal activities, and driv

In [5]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

# Setup quantization arguments.
#   We also support speedup from 8 bits.
#   With 8 bits, its best to set group_size=-1 (channelwise) / desc_act=False)
quantize_config = BaseQuantizeConfig(
    bits=4,                         # Only support 4 bit
    group_size=128,                 # Group size 128 is typically the best spot for accuracy / performance.
    desc_act=True,                  # Act_recordering will help accuracy.
    model_file_base_name="model",   # Name of the model.safetensors when we call save_pretrained
    true_sequential=False,          # Set to true to reduce memory consumption for slower runtime of .quantize()
)

# Load model.
model = AutoGPTQForCausalLM.from_pretrained(
    MODEL_ID,
    quantize_config,
    device_map="auto")

CUDA extension not installed.
CUDA extension not installed.
/home/rshaw/.pyenv/versions/3.10.14/envs/auto-gptq/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.01it/s]


In [6]:
# Apply the GPTQ algorithm.
model.quantize(examples)

INFO - Start quantizing layer 1/32
INFO - Quantizing self_attn.k_proj in layer 1/32...
INFO - Quantizing self_attn.v_proj in layer 1/32...
INFO - Quantizing self_attn.q_proj in layer 1/32...
INFO - Quantizing self_attn.o_proj in layer 1/32...
INFO - Quantizing mlp.up_proj in layer 1/32...
INFO - Quantizing mlp.gate_proj in layer 1/32...
INFO - Quantizing mlp.down_proj in layer 1/32...
INFO - Start quantizing layer 2/32
INFO - Quantizing self_attn.k_proj in layer 2/32...
INFO - Quantizing self_attn.v_proj in layer 2/32...
INFO - Quantizing self_attn.q_proj in layer 2/32...
INFO - Quantizing self_attn.o_proj in layer 2/32...
INFO - Quantizing mlp.up_proj in layer 2/32...
INFO - Quantizing mlp.gate_proj in layer 2/32...
INFO - Quantizing mlp.down_proj in layer 2/32...
INFO - Start quantizing layer 3/32
INFO - Quantizing self_attn.k_proj in layer 3/32...
INFO - Quantizing self_attn.v_proj in layer 3/32...
INFO - Quantizing self_attn.q_proj in layer 3/32...
INFO - Quantizing self_attn.o_pro

In [7]:
gptq_save_dir = f"{MODEL_ID.split('/')[-1]}-GPTQ"
print(f"Saving gptq model to {gptq_save_dir}")
model.save_pretrained(gptq_save_dir)
tokenizer.save_pretrained(gptq_save_dir)

WARNING - you are using save_pretrained, which will re-direct to save_quantized.


Saving gptq model to Meta-Llama-3-8B-Instruct-GPTQ


('Meta-Llama-3-8B-Instruct-GPTQ/tokenizer_config.json',
 'Meta-Llama-3-8B-Instruct-GPTQ/special_tokens_map.json',
 'Meta-Llama-3-8B-Instruct-GPTQ/tokenizer.json')